# Meeting minutes creator

In this norebook, we make a video minutes program using huggingface transformers.



In [1]:
import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
from dotenv import load_dotenv

In [2]:

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [3]:
# Sign in to HuggingFace Hub
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY')

login(huggingface_api_key, add_to_git_credential=True)

In [4]:
openai = OpenAI(api_key=openai_api_key)

In [5]:
from pytube import YouTube
from pytubefix import YouTube
from pytubefix.cli import on_progress

In [6]:

def youtube_audio_downloader(video_url):
    # Replace with your YouTube video URL
    
    # Get YouTube video object
    yt = YouTube(video_url,use_oauth=True, allow_oauth_cache=True)
    
    # Filter only audio streams
    audio_stream = yt.streams.filter(only_audio=True).all()
    #print(audio_stream)
    #stream = yt.streams.get_by_itag(140)
    download_path = audio_stream[0].download()
    return download_path 

In [7]:
def audio2transcipt(download_path):
    audio_file = open(download_path, "rb")
    transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
    return transcription

In [8]:
transcription = audio2transcipt(youtube_audio_downloader("https://www.youtube.com/watch?v=tinKv_mSw8E"))

C:\Users\benny\AppData\Local\Temp\ipykernel_40008\3571359578.py:8: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  audio_stream = yt.streams.filter(only_audio=True).all()


In [9]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a youtube video. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [10]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16, 
    bnb_4bit_quant_type="nf4"
)

In [11]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a youtube video. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
side.<|eot_id|><|start_header_id|>assistant<|end_header_id|>ust a few minutes. It's a simple framework that changed my life and my work forever. It's called the Rule of 300. I first learned this rubbing shoulders with literally some of the smartest and most successful billionaires you probably know. And I've personally used this rule to create my dream life, including speaking on Tony Robbins stage, flying around in my own jet, and mentoring hundreds of kids

In [12]:
response = tokenizer.decode(outputs[0])

In [15]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a youtube video. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
It took me 15 years to learn what I'm about to tell you in just a few minutes. It's a simple framework that changed my life and my work forever. It's called the Rule of 300. I first learned this rubbing shoulders with literally some of the smartest and most successful billionaires you probably know. And I've personally used this rule to create my dream life, including speaking on Tony Robbins stage, flying around in my own jet, and mentoring hundreds of kids in my local community. So without further explaining it, this is the Rule of 300. Starting with 100% clarity. The crazy thing I've realized is that most people can't tell you what they want. I hike with entrepreneurs every week, and I'm always asking them about their dreams, their goals. Most of them cannot tell me what they want to create. They can't tell me what they want to achieve, what they want to buy, what they want their life to stand for. Versus you got a guy like Sam on my team who's crystal clear about what he wants to do, how he wants to show up, and what he wants to achieve. And what I've learned is when people are clear about what they want, it allows other people to help them. That's why I felt super privileged that I could support him by helping him buy his dream car, his GT4. How are you supposed to know what somebody wants if they don't tell you? How is anybody supposed to help you if you don't tell them? Everybody knows Gary Vaynerchuk wants to buy the New York Jets. Why? Because he tells everybody. He's got clarity around it. Elon Musk wants to colonize Mars. He says it every time he's in front of a mic, every time there's a camera on him. He says it. Our mission is, our mission is, our mission is. And you get a bunch of people doing that. I'm on a mission to build a billion-dollar software company. That is my mission. It's what I've always said. It's what I've always wanted to do. I just keep telling people. I tell my team. And when you activate it by being clear, other people can support you. We don't create our future, we attract it. See, most people think I got to do all this stuff to eventually get something. No, you have to become the person to attract into your life those results. So how do you find clarity? Number one, dream big. If I could go back to talk to my 20-year-old self, I'm telling you, I would just say bigger, bigger, bigger. You underestimate what's possible when you focus and you compound over one year, five years, 10 years, 25 years. So start with a one-year goal, but then build the five and the 55. I've got a business plan for 55 years into the future. The reason why is because I know that I'm going to wake up every day and create. So make sure you're dreaming big. Number two is shop your dreams. I'm a big fan of encouraging people to go stay in a dream house, go rent a supercar. If you can't afford it, go sit in the lobby at the Ritz-Carlton, go shop your dreams, get around the energy, the smell. See, most people just don't allow themselves to touch their dream. And the truth is, is that's the only way that you're going to make it normal. You're going to create a sense of this is normal. A few months ago, I was talking to one of my sales guy. His dream was to drive a Porsche. I said, dude, have you visited the Porsche dealership? Have you sat in your dream car? Have you asked the sales guy to run the numbers to let you know even what it would cost? He's like, no, get in my car. So we drove to the dealership. Not only did he sit in it, I told the sales guy, send him the numbers. How much is the down payment? What it's going to cost per month? Why? Get familiar with it. When I decided to start flying private, I got familiar with it. I got surrounded with people. I wanted to understand it. I wanted to figure out the numbers. I wanted to see what would need to be true for it to make sense in my life. Most people don't allow themselves to even shop their dreams. Three is write them down. If you don't write it down, you won't remember it. If you don't write it down, you won't commit to it. If you don't write it down, you're not being specific enough. You're more likely to achieve goals that are written down. This is scientifically backed by all the research around goal setting. Write it down. I have a notes file on my phone, okay? It's called 12 Power Goals. On that, I have my 12 goals for the year. I review that three to four times a day. Why? I want to keep it front and center. Think about it for yourself. If you just review your dreams, your goals every day, especially the ones for the next 12 months, it'll make it inevitable that you'll achieve that. Most people will never do it. Be the person that's willing to do the thing that nobody else is doing to achieve the goals. Number four is create a vision board. I'm a big fan of visualizing what you want to achieve. Why? Your brain doesn't think in words. It thinks in pictures. We have this thing called the reticular activating system. When we give it a picture to scan the world, it will literally ignore all other information and just find the thing that's going to help you achieve your goal. If you get clear on what you want to achieve, you will make success inevitable, which brings us to 100% belief. This one is the hard one. Your beliefs either hold up your vision, but sometimes beliefs can actually hold you back from what you want to achieve. For example, number one, limiting beliefs. Limiting beliefs are around you, about your skills. Maybe you might think I'm not smart enough. I don't have the right network or connections or it takes money to make money. All of these things are limiting beliefs about your possibility. Here's what's crazy about limiting beliefs. You'll never hold on to what you don't think you deserve. You will quickly get rid of it, even if you accidentally get it. Number two is negative beliefs. These are beliefs about the world, about how everybody else is. If you think the world is bad, you will always find bad. If you think the world is positive, you'll find positive. Most people talk crap about other people that are successful so that they don't have to become successful. If I think that the person driving the supercar is an a-hole, then why would I want to have a car? Because then everybody's going to think I'm an a-hole. But the truth is your negative belief around how the world is, it's causing you to play small. If you love people and you want to help them, the number one thing you can do is inspire them to dream bigger. And playing small never helped anybody. Number three is positive beliefs. Positive beliefs as in you've already been there. You've done it, it's accomplished, you got the t-shirt. You want to get into the headspace of beliefs where what you're trying to achieve is very familiar to you. Talk as if it's already happened. I remember watching on TikTok recently a reel about Conor McGregor talking on a mic about his achievements. And he pretty much said, if you can see it and you speak it, you can achieve it. I think he was talking to Ronaldo, the soccer player. He's asking him about his jet. Have you got a jet yet? No? Yeah, I might be. I want to get a jet. We're talking like young Conor McGregor that hadn't achieved anything. He saw it, he spoke it, and he said, if you can do those things, you will achieve. See, everything that's ever existed in the whole world, this desk, this mic, this camera, was first seen in the mind of somebody from the past. Isn't that wild? Your whole life, your whole existence is essentially living in the dreams of people from the past because nothing's ever got created, existed, ever before somebody saw it in their mind. That is how it's always been. What I need you to do is believe that it's possible for you. Sure, it's going to take work. Sure, it's going to take effort. Sure, it's going to take resources. It may not happen overnight. It won't happen overnight. But I need you to believe that it's yours to have. You wouldn't have it on your heart. You wouldn't desire it if you couldn't achieve it. That I believe. If you can hold a belief true, you will make success inevitable. Which brings us to 100% of the time. See, when a plane takes off, most people don't realize it, but most of the fuel required to fly from point A to point B over an hour is used in that first takeoff. That is momentum. If you don't believe 100% of the time, if you don't keep that clarity and that belief there, you're not going to build the momentum. The person that writes the business plan that opens it up in six months or review it, that's not a winner. People that win in line, not only hold the belief, have clarity, but they do that as frequently as possible. Think about somebody that's had a big weight loss journey. They have looked at their weight, thought about their meals, shared their preferences with their friends, talk about it nonstop. 100% of the time, they have a vision for what's possible. I'm losing the weight. I'm going to look like this. And they don't waver from it. Momentum isn't about perfect days. It's about showing up on imperfect ones. So the rule of 300, how can you keep 100% clarity on your goals? 100% belief that you can achieve it 100% of the time so you don't waver. The way I do it is a few steps. The first one is I design my environment. See, I want to default the design of my space, of my decisions to be already made to win. Putting my clothes out so that I work out in the morning, making sure my peer group of people are folks closer to my goals, not closer to me, making sure that I curate the space to win. That's number one. Number two, you have to manage your energy. See, we don't manage our time. Time comes and goes, you can't stop that. Can you manage your energy? Can somebody that's high energy and positive achieve more than somebody's low energy and negative? Just think about that feeling. If I said, hey man, show me what a negative energy person looks like, you'd be like, you know, head down, shoulders forward, low energy, nobody wants to help that person. If I say high energy, you go, ooh. All of a sudden, just what you eat, your exercise, these are things that manage your energy is going to help you get there faster. The third is build your achievement list. When you feel down, when you feel defeated, when you feel like a setback or you're going into a big moment in your life, make a list of all the achievements you've ever done that's allowed you to feel proud. Because I want you to remind yourself how great you are. See, you wouldn't be here if you didn't believe that you were destined for greatness. I know the fact that you're here in my voice right now tells me that you are gonna do something awesome with your life. Because of that, I want you to make that list. That list of things, small things, like first time for me, I ever did a kickflip on a skateboard. Now, I read that list, it's like, oh, sold my first company at 28. That's crazy. And then did it again at 30. And then did it again at 32. It's wild. All these things that have happened. Bought my first jet. This is crazy. But I review these things to remind me about what I've accomplished. Because in those moments of doubt, reviewing that list will get me out of it. It doesn't matter how small it is. It's about the energy and remembering what you've accomplished in the past. The reason why this list works so well is because in those moments where you feel defeated or you don't feel enough, or you've got to show up in a powerful way in a critical meeting, reviewing that list will remind you that you've already done the work. You've been there before. You've had those setbacks and you've overcome them. And that is the energy that you need to remind yourself that the belief is there, that you've had the clarity before, and that you've done hard things. And that'll change the game for you. The 300% rule will make your success inevitable. That's the process. We have to show up with clarity, belief, and time to make your goals a reality. But if you want to learn the eight habits that will fix 98% of your problems, click the video and I'll see you on the other side.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Meeting Minutes**
====================
### Summary

* Date: Today (26 Jul 2024)
* Location: Virtual Meeting
* Attendees: [List of attendees]
* Summary: This meeting discussed the importance of clarity, belief, and momentum in achieving goals. The speaker shared his personal experiences and the "Rule of 300" framework to help individuals create their dream life.

### Discussion Points

* The importance of starting with 100% clarity and being clear about what you want to achieve
* The role of limiting beliefs, negative beliefs, and positive beliefs in holding back or propelling individuals towards their goals
* The need to manage energy and create an environment that supports success
* The importance of building an achievement list to remind oneself of past accomplishments and boost confidence
* The connection between momentum and achieving goals

### Takeaways

* Clarity is essential for achieving goals and creating a dream life
* Beliefs can either hold you back or propel you towards your goals
* Managing energy and creating an environment that supports success is crucial
* Building an achievement list can help individuals stay motivated and confident
* Momentum is key to achieving goals and making success inevitable

### Action Items

* **Owner: [Name]**
	+ Review and implement the "Rule of 300" framework to create a clear vision for their goals
	+ Identify and challenge limiting beliefs that may be holding them back
	+ Create an environment that supports success and manage their energy effectively
	+ Build an achievement list to remind themselves of past accomplishments and boost confidence
* **Owner: [Name]**
	+ Share the "Rule of 300" framework with their team or community to help others achieve their goals
	+ Encourage individuals to start with 100% clarity and be clear about what they want to achieve
	+ Emphasize the importance of managing energy and creating an environment that supports success<|eot_id|>